In [15]:
import numpy as np
import pandas as p
import pickle

promoted_content = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/promoted_content.csv", usecols = ["ad_id", "document_id"], dtype = {"ad_id":int, "document_id":int})
ad_on_doc = p.read_csv("/Users/astrachan/Desktop/outbrain/output_tables/document_on_ad.csv", usecols = ["ad_id", "document_id", "score"], dtype={"document_id": int, "ad_id":int, "score": float})
documents_categories = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_categories.csv", dtype = {"document_id":int, "category_id":int, "confidence_level":float})
#documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_topics.csv", dtype = {"document_id":int, "topic_id":int, "confidence_level":float})
#train = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/train.csv", usecols = ["display_id", "ad_id", "clicked"], dtype={"display_id": int, "ad_id": int, "clicked" : bool})
#events = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/events.csv", usecols = ["display_id", "document_id"], dtype={"display_id": int, "document_id" : int})
#train = train.merge(events, how = 'left', on = 'display_id')
#del events

In [16]:
#remove category ids below a certain confidence level and merge promoted with documents_categories_reduced
confidence_cut = 0.4
documents_categories_reduced = documents_categories[documents_categories['confidence_level'] > confidence_cut]
print 'category ids precent: ' + repr(float(len(documents_categories_reduced.category_id.unique())) / len(documents_categories.category_id.unique()))
print 'unique category ids: ' + repr(len(documents_categories_reduced.category_id.unique()))
promoted = promoted_content.merge(documents_categories_reduced, how='left', on='document_id').drop('document_id', axis=1)

category ids precent: 0.979381443298969
unique category ids: 95


In [27]:
#take only highest scored ad_on_doc
score_cut = 1000 #which portion of top scores to take
ad_on_doc_reduced = ad_on_doc[:ad_on_doc.shape[0] / score_cut]
print ad_on_doc_reduced.shape
print 'precentage taken: ' + repr(float(ad_on_doc_reduced.shape[0]) / ad_on_doc.shape[0])
print 'minimal score taken: ' + repr(ad_on_doc_reduced.score.min())

(26891, 3)
precentage taken: 0.0009999866499867114
minimal score taken: 2.6709886333299999


In [18]:
#merge with category ids of doc and ad
ad_on_doc_reduced = ad_on_doc_reduced.merge(documents_categories_reduced, how='left', on='document_id')
ad_on_doc_reduced = ad_on_doc_reduced.merge(promoted, how='left', on='ad_id', suffixes=('_doc','_ad'))

In [19]:
#drop lines with NA and lines with same category on both ad and doc, remove confidence, ad_id and doc_id columns
ad_on_doc_reduced = ad_on_doc_reduced.dropna()
#ad_on_doc_reduced = ad_on_doc_reduced[ad_on_doc_reduced.category_id_ad != ad_on_doc_reduced.category_id_doc]
ad_on_doc_reduced.drop(ad_on_doc_reduced.columns[[0,1,4,6]],axis = 1,inplace=True)
ad_on_doc_reduced.reset_index(drop=True, inplace=True)

In [20]:
print ad_on_doc_reduced.shape
print len(ad_on_doc_reduced.category_id_doc.unique())
print len(ad_on_doc_reduced.category_id_ad.unique())

(20334, 3)
87
87


In [21]:
ad_on_doc_reduced.head()
#consider using scores for calculating correlation later

,score,category_id_doc,category_id_ad
0,7.511080,1403.0,1403.0
1,7.189740,1607.0,1506.0
2,6.974017,1403.0,1506.0
3,6.950940,1808.0,1406.0
4,6.745651,1607.0,1610.0


In [22]:
#put lower category_id of both on left side
for i, row in enumerate(ad_on_doc_reduced.itertuples()):
    if row.category_id_doc > row.category_id_ad:
        ad = row.category_id_ad
        doc = row.category_id_doc
        ad_on_doc_reduced.set_value(i,'category_id_doc', ad)
        ad_on_doc_reduced.set_value(i,'category_id_ad', doc)
ad_on_doc_reduced.rename(columns={'category_id_doc' : 'category_id_l', 'category_id_ad' : 'category_id_r'},inplace=True)

In [23]:
#count how many times each pair shows, and reduce by number of shows
#keep top 20% score even if count is 1
high_score = ad_on_doc_reduced.score.quantile(q=0.8, interpolation='higher')
ad_doc_count = ad_on_doc_reduced.groupby(['category_id_l','category_id_r']).score.agg({'mean_score' : 'mean', 'count' : 'count'}).reset_index()
ad_doc_count = ad_doc_count[(ad_doc_count['count'] > 1) | (ad_doc_count['mean_score'] > high_score)]
print ad_doc_count.head()
print len(ad_doc_count.category_id_l.unique())
print len(ad_doc_count.category_id_r.unique())
print ad_doc_count.mean_score.mean()
print ad_doc_count.mean_score.max()

   category_id_l  category_id_r  count  mean_score
2         1000.0         1403.0      3    3.088400
3         1000.0         1405.0      2    2.892626
5         1000.0         1504.0      4    2.958595
6         1000.0         1505.0      5    2.923549
7         1000.0         1510.0      1    3.223962
82
83
2.98619899744
5.33096828494


In [24]:
#create a dictionary, for each tuple (x_category,y_category) where x_category < y_category return the mean_score
dict_category = {}
for row in ad_doc_count.itertuples():
    key = (row.category_id_l,row.category_id_r)
    score = row.mean_score
    dict_category[key] = score

In [28]:
#save dictionary to file
output = open('/Users/astrachan/Desktop/outbrain/dicts/dict_category_' + repr(confidence_cut) + '_' + repr(score_cut), 'ab+')
pickle.dump(dict_category, output)
output.close()

In [25]:
#check categories that 'sleep arround' with all other categories, they make it so we have very few sets
#eventually didn't use, also the rest of the code
reduce_dominants_left = ad_doc_count.groupby('category_id_l').category_id_r.count().reset_index(name = 'count')
reduce_dominants_left.sort_values('count',inplace=True,ascending=False)
reduce_dominants_right = ad_doc_count.groupby('category_id_r').category_id_l.count().reset_index(name = 'count')
reduce_dominants_right.sort_values('count',inplace=True,ascending=False)
#arrange to one reduce_dominants table
print reduce_dominants_left.head(10)
print reduce_dominants_right.head(10)
reduce_dominants_left.rename(columns={'category_id_l' : 'category_id'},inplace=True)
reduce_dominants_right.rename(columns={'category_id_r' : 'category_id'},inplace=True)
reduce_dominants = p.merge(reduce_dominants_left,reduce_dominants_right,how='outer',on='category_id')
reduce_dominants['count'] = reduce_dominants.count_x + reduce_dominants.count_y
reduce_dominants.drop(reduce_dominants.columns[[1,2]],axis = 1,inplace=True)
reduce_dominants.sort_values(by = 'count',inplace=True,ascending=False)
print reduce_dominants.head(10)
print reduce_dominants.shape

    category_id_l  count
1          1100.0     68
19         1403.0     65
5          1205.0     59
12         1302.0     49
23         1408.0     42
21         1406.0     41
10         1210.0     34
26         1503.0     32
20         1405.0     31
50         1702.0     30
    category_id_r  count
78         2003.0     69
82         2100.0     55
79         2004.0     49
54         1708.0     40
48         1702.0     38
66         1903.0     36
60         1805.0     35
63         1808.0     35
52         1706.0     33
44         1611.0     30
    category_id  count
1        1403.0   83.0
65       2003.0   73.0
0        1100.0   69.0
9        1702.0   68.0
17       1708.0   65.0
2        1205.0   62.0
4        1408.0   60.0
3        1302.0   58.0
81       2100.0   56.0
5        1406.0   55.0
(87, 2)


In [902]:
dict_category = {}
for i, row in enumerate(ad_doc_count.itertuples()):
    key = row.category_id_l
    item = row.category_id_r
    if key not in dict_category:
        dict_category[key] = set([item,key])
    else:
        dict_category[key] = dict_category[key] | set([item])

In [903]:
print len(dict_category)
for x in dict_category:
    print dict_category[x]

134
set([16.0, 0.0, 259.0, 68.0, 288.0, 175.0])
set([160.0, 1.0])
set([127.0, 184.0, 3.0, 93.0, 183.0])
set([160.0, 240.0, 66.0, 4.0, 184.0, 140.0, 127.0])
set([184.0, 6.0])
set([160.0, 8.0])
set([131.0, 198.0, 227.0, 10.0, 140.0, 270.0])
set([16.0, 11.0, 18.0, 51.0, 289.0])
set([259.0, 12.0])
set([250.0, 13.0])
set([160.0, 140.0, 198.0, 15.0])
set([66.0, 131.0, 198.0, 136.0, 140.0, 16.0, 148.0, 277.0, 36.0, 154.0, 175.0, 26.0, 95.0, 160.0, 97.0, 283.0, 164.0, 229.0, 102.0, 234.0, 255.0, 46.0, 93.0, 250.0, 127.0, 252.0, 21.0])
set([17.0, 132.0, 271.0])
set([18.0, 51.0])
set([296.0, 19.0])
set([97.0, 66.0, 20.0, 102.0, 184.0, 156.0])
set([160.0, 184.0, 259.0, 21.0, 183.0])
set([296.0, 227.0, 24.0, 105.0, 107.0, 156.0, 271.0])
set([64.0, 130.0, 198.0, 262.0, 107.0, 147.0, 26.0, 283.0, 156.0])
set([160.0, 28.0])
set([156.0, 140.0, 30.0])
set([160.0, 33.0, 140.0])
set([160.0, 66.0, 227.0, 36.0, 198.0, 103.0, 232.0, 140.0, 183.0, 287.0, 252.0, 93.0, 127.0])
set([160.0, 140.0, 69.0, 37.0])
s

In [904]:
k = 1

while (k !=0):
    t = 1
    for x in dict_category:
        for y in dict_category:
            if x != y:
                if ((dict_category[x] & dict_category[y]) != set([])):
                    dict_category[x] = dict_category[x] | dict_category[y]
                    dict_category.pop(y)
                    t = 0
                    break
        if t == 0:
            break
    if t != 0:
        k = 0

In [905]:
print len(dict_category)

1


In [906]:
for x in dict_category:
    print dict_category[x]

set([0.0, 1.0, 3.0, 4.0, 6.0, 8.0, 10.0, 11.0, 12.0, 13.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 24.0, 26.0, 28.0, 30.0, 33.0, 36.0, 37.0, 38.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 51.0, 52.0, 54.0, 55.0, 61.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 73.0, 74.0, 75.0, 80.0, 81.0, 83.0, 84.0, 85.0, 88.0, 89.0, 93.0, 94.0, 95.0, 96.0, 97.0, 100.0, 101.0, 102.0, 103.0, 105.0, 107.0, 108.0, 111.0, 112.0, 115.0, 116.0, 117.0, 118.0, 119.0, 122.0, 124.0, 125.0, 127.0, 128.0, 130.0, 131.0, 132.0, 136.0, 137.0, 138.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 147.0, 148.0, 150.0, 151.0, 154.0, 156.0, 157.0, 160.0, 164.0, 165.0, 167.0, 168.0, 169.0, 170.0, 174.0, 175.0, 177.0, 182.0, 183.0, 184.0, 185.0, 186.0, 187.0, 188.0, 189.0, 193.0, 197.0, 198.0, 199.0, 200.0, 201.0, 202.0, 203.0, 204.0, 205.0, 207.0, 208.0, 211.0, 212.0, 214.0, 215.0, 216.0, 219.0, 221.0, 222.0, 227.0, 229.0, 232.0, 234.0, 236.0, 237.0, 238.0, 239.0, 240.0, 241.0, 248.0, 249.0, 250.0, 252.0, 253.0, 255.